In [10]:
from pymongo import MongoClient
from collections import Counter

In [11]:
uri = "mongodb://localhost:27017"
db_name = "mtg_decks_db"
client = MongoClient(uri)
db = client[db_name]

In [12]:
pipeline = [
    {"$match": {"isSanitized": True}},
    {"$project": {"all_cards": {"$concatArrays": [
        "$maindeck", "$sideboard"]}, "name": 1}},
    {"$unwind": "$all_cards"},
    {"$group": {"_id": {"_id": "$_id", "card_name": "$all_cards",
                        "name": "$name"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id._id", "name": {"$first": "$_id.name"},
                "cards": {"$push": {"name": "$_id.card_name", "count": "$count"}}}}
]

decks = list(db.decks.aggregate(pipeline))
decks

[{'_id': ObjectId('6614a907bc0fae7824b1ca09'),
  'name': 'Kuldotha Red',
  'cards': [{'name': 'Experimental Synthesizer', 'count': 4},
   {'name': 'Chain Lightning', 'count': 3},
   {'name': 'End the Festivities', 'count': 2},
   {'name': 'Implement of Combustion', 'count': 4},
   {'name': 'Searing Blaze', 'count': 4},
   {'name': 'Fireblast', 'count': 2},
   {'name': 'Goblin Bushwhacker', 'count': 4},
   {'name': 'Goblin Grenade', 'count': 2},
   {'name': "Wrenn's Resolve", 'count': 3},
   {'name': 'Goblin Tomb Raider', 'count': 4},
   {'name': 'Smash to Smithereens', 'count': 4},
   {'name': 'Pyroblast', 'count': 3},
   {'name': 'Lightning Bolt', 'count': 4},
   {'name': 'Cast into the Fire', 'count': 1},
   {'name': 'Reckless Impulse', 'count': 2},
   {'name': 'Flaring Pain', 'count': 1},
   {'name': 'Kuldotha Rebirth', 'count': 4},
   {'name': 'Mountain', 'count': 13},
   {'name': 'Galvanic Blast', 'count': 4},
   {'name': 'Voldaren Epicure', 'count': 4},
   {'name': 'Great Furnace

In [13]:
personal_pool = {}
for card in db.personal_pool.find({}, {"name": 1, "count": 1, "_id": 0}):
    personal_pool[card["name"]] = card["count"]

personal_pool

{'_____ Goblin': 4,
 'Abrade': 4,
 'Acorn Harvest': 4,
 'All That Glitters': 4,
 'Alpine Meadow': 4,
 'Ancient Den': 4,
 'Ancient Stirrings': 4,
 'Annul': 4,
 "Apostle's Blessing": 4,
 'Arboreal Grazer': 3,
 'Archaeomancer': 4,
 'Architects of Will': 4,
 'Arctic Treeline': 3,
 'Ardent Elementalist': 4,
 'Arms of Hadar': 4,
 'Ash Barrens': 4,
 'Augur of Bolas': 4,
 'Aura Flux': 1,
 'Axebane Guardian': 5,
 'Azorius Chancery': 4,
 'Azure Fleet Admiral': 4,
 'Barren Moor': 4,
 'Behold the Multiverse': 5,
 'Benevolent Bodyguard': 5,
 'Bladeback Sliver': 2,
 'Blood Celebrant': 4,
 'Blood Fountain': 4,
 'Blue Elemental Blast': 4,
 'Bojuka Bog': 4,
 'Boulderbranch Golem': 4,
 'Brainstorm': 4,
 'Breath Weapon': 4,
 'Brinebarrow Intruder': 4,
 'Bring the Ending': 4,
 'Brute Force': 3,
 'Burning Prophet': 4,
 'Cabal Ritual': 4,
 'Cast Down': 4,
 'Cast into the Fire': 4,
 'Cave of Temptation': 1,
 'Chain Lightning': 4,
 'Chromatic Sphere': 4,
 'Chromatic Star': 4,
 'Compulsive Research': 2,
 'Cont

In [14]:
usable_decks = []
for deck in decks:
    can_build = True
    for card in deck["cards"]:
        if card["name"] not in personal_pool or personal_pool[card["name"]] < card["count"]:
            can_build = False
            break
    if can_build:
        usable_decks.append(deck["_id"])

In [15]:
def get_decklist_as_text(decklist):
    card_counts = Counter(decklist)
    card_dict = dict(card_counts)

    formatted_decklist = [f"{value} {key}" for key, value in card_dict.items()]
    formatted_decklist.sort()

    return "\n".join(formatted_decklist)


def get_deck_as_text(id):
    deck = db.decks.find_one({"_id": id})
    # deck = db.decks.find_one({"_id": id, "in_moxfield": {"$ne": True}})

    if deck:
        maindeck = get_decklist_as_text(deck["maindeck"])
        sideboard = get_decklist_as_text(deck["sideboard"])

        db.decks.update_one({"_id": id}, {"$set": {"in_moxfield": True}})
        return f"{deck["name"]} ({deck["_id"]})\n{maindeck}\n\n{sideboard}\n\n\n\n"


for deck_id in usable_decks:
    with open(f"../decks.txt", "a") as file:
        file.write(get_deck_as_text(deck_id))